In [53]:
import pandas as pd
import numpy as np

In [54]:
def get_train_data():
    return pd.read_csv("train.csv")

def get_test_data():
    test_comments = pd.read_csv("test.csv")
    test_labels = pd.read_csv("test_labels.csv")
    return test_comments, test_labels

In [55]:
training_data = get_train_data()
test_comments, test_labels = get_test_data()
test_data = test_comments.set_index('id').join(other=test_labels.set_index('id'))
test_data = test_data[test_data.toxic != -1]

In [64]:
training_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [65]:
num_toxic = len(training_data.loc[training_data.toxic == 1])
num_severe = len(training_data.loc[training_data.severe_toxic == 1])
num_obscene = len(training_data.loc[training_data.obscene == 1])
num_threat = len(training_data.loc[training_data.threat == 1])
num_insult = len(training_data.loc[training_data.insult == 1])
num_hate = len(training_data.loc[training_data.identity_hate == 1])

In [73]:
print(num_toxic, num_severe, num_obscene, num_threat, num_insult, num_hate)

15294 1595 8449 478 7877 1405


In [74]:
print(num_toxic/len(training_data), num_severe/len(training_data), num_obscene/len(training_data), num_threat/len(training_data), num_insult/len(training_data), num_hate/len(training_data))

0.09584448302009764 0.009995550569965721 0.052948217407925 0.002995531769557125 0.04936360616904074 0.00880485802558109
